In [1]:
#read all files into an array (dataframes)
import numpy as np
import os
import pandas as pd
from IPython.display import display
import datetime
import re
import json
directory = os.getcwd() 

def filter_one_or_zero(n):
  return 1 if (n) else 0 

def split_wares(wares):
  res = re.sub(r".*wares\:", "", str(wares))
  return json.loads(res)

def read_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("Buildingslogs.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=[0],skiprows=[1]))
  return dataframes


#group the data by building where d[str(serial)] = group

def group_data_by_building(data):
  d = {}
  for data in data :
    for serial, group in data.groupby('serial'):
      d[str(serial)] = group
  return d


def int_to_time(input,hours_mapping):
  return datetime.datetime(2021, 1, 1) + datetime.timedelta(hours=input/hours_mapping)

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +datetime.timedelta(hours=t.minute//30))  

def map_to_real_time(data):
  # hou much is an hour in game time 
  # for example 1hour = 2000 game time unite
  res=data
  hour_in_numbers = (int(data[0]["time"].max()) - int(data[0]["time"].min())) / (365*24)
  for df in res :
    df["time"]=df["time"].apply(int_to_time,args=[hour_in_numbers])
    #df["time"]=df["time"].apply(hour_rounder)
    df.time=df.time.round("0.1S")
  return res  

def read_orders_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("orderslogs.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=[0],skiprows=[1]))
  return dataframes

#get one buildings stockes according to IBP Stock
#data_line is one line where wares != nan of the original data
def data_line_to_one_building_stock_IBP(position):
  #locations = pd.DataFrame(columns=['LOCNO','LOCTYPE','NAME1','NAME2','LAND1','REGIO','ORT01','PSTLZ','PSTL2','STRAS','LOEVM','TIME_ZONE','TELF1','FABKL','KUKLA','NODETYPE','CUST_ATTR1','CUST_ATTR2','CUST_ATTR3','CUST_ATTR4','CUST_ATTR5','CUST_ATTR6','CUST_ATTR7','CUST_ATTR8','CUST_ATTR9','CUST_ATTR10','LONGITUDE','LATITUDE'])
  #stocks = pd.DataFrame(columns=['MATNR','LOC_WERKS','VRFKZ','MNG01'])
  locatins=["0"]*4
  locatins[0]=position
  locatins[1]="V"
  locatins[2]="CET"
  locatins[3]="03"
  #locations["LOCNO"]=position
  #locations["LOCTYPE"]="V"
  #locations["TIME_ZONE"]="CET"
  #locations["FABKL"]="03"
  return locatins
  #group_data_by_building

def write_series_of_location_to_csv_as_IBPlocation(locationseries_nameseries,filename):

  IBPlocations = pd.DataFrame(columns=['LOCID','GEOLATITUDE','GEOLONGITUDE','LOCATIONDEL','LOCATIONREGION','LOCATIONTYPE','LOCCITY','LOCCOUNTRY','LOCNAME','LOCNAME2','LOCTIMEZONE','PHONE','PLANTTYPE','POBOXZIP','PRODUCTIONCALID','STREET','ZIP'])
  IBPlocations[["LOCID","LOCNAME"]]=locationseries_nameseries
  
  #x = set([x[0] for x in test["position"]])
  #locations["'LATITUDE'"]=list(x)
  #IBPlocations["MANDT"]="001"
  IBPlocations["LOCATIONDEL"]=" "
  #IBPlocations["LOGSYS"]="WDL"
  IBPlocations["LOCATIONTYPE"]="P"
  IBPlocations["LOCTIMEZONE"]="CET"
  IBPlocations["PRODUCTIONCALID"]="01"
  IBPlocations["PLANTTYPE"]="PL"
  #IBPlocations.drop_duplicates(subset ="LOCNO",keep = False, inplace = True)
  IBPlocations.to_csv("locations"+filename+ ".csv",sep=";",index=False,header=True)

def write_singel_string_wares_attribute_to_csv_as_IBPmaterial(wares):
  IBPmaterial = pd.DataFrame(columns=['MANDT','LOGSYS','MATNR','LVORM','MATKL','MEINS','PRDHA','MTART','CUST_ATTR1','CUST_ATTR2','CUST_ATTR3','CUST_ATTR4','CUST_ATTR5','CUST_ATTR6','CUST_ATTR7','CUST_ATTR8','CUST_ATTR9','CUST_ATTR10'])

  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())

  wares_amount_data.columns = ['ware', 'amount']
  IBPmaterial["MATNR"]=wares_amount_data["ware"]
  IBPmaterial["MEINS"]="piece"
  IBPmaterial["MANDT"]="001"
  IBPmaterial["LOGSYS"]="WDL"
  IBpmaterialdes = pd.DataFrame(columns=["MANDT","LOGSYS","RESID","SPRAS","KTEXT"])
  IBpmaterialdes["RESID"]=IBPmaterial["MATNR"]
  IBpmaterialdes["SPRAS"]="E"
  IBpmaterialdes["MANDT"]="001"
  IBpmaterialdes["LOGSYS"]="WDL"
  IBpmaterialdes.to_csv("materialsdesc.csv",sep=";",index=False,header=False)
  IBPmaterial.to_csv("materials.csv",sep=";",index=False,header=False)


def write_product_location(building_entry,filename):
  IBPproduct_location = pd.DataFrame(columns=["LOCID","PRDID","ASSEMBLYSCRAP","DEMANDSUBSTTYPE","DETPLANNINGSCHEDULINGHORIZON",
  "FAIRSHAREEXCESSSTOCKSEGMENT","FAIRSHAREMAXSTOCKSEGMENT","FAIRSHARESAFETYSTOCKSEGMENT","FORECASTCONSPROFILE",
  "FREEZEHORIZONDISTRIBUTION","FREEZEHORIZONPRODUCTION","GOODSRECEIPTPROCTIME","INBOUNDCALID",
  "LOCATIONPRODUCTDEL","MRPCONTROLLER","PLANDELETIONHORIZON","PLANNEDDELIVERYTIME","PMAXLOTSIZE",
  "PMINLOTSIZE","PPINTEGRATIONMODE","PROCUREMENTTYPE","PRODUCTALLOCATIONPROCEDURE","PROUNDING",
  "SAFETYSTOCKMD","SHIPPINGCALID","SOURCEOFSUPPLYSELPROFILE","SUBNETWORKID"])
  wares=building_entry["wares"]
  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())
  wares_amount_data.columns = ['ware', 'amount']
  IBPproduct_location["PRDID"]=wares_amount_data["ware"]
  #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
  IBPproduct_location['LOCID']=str(building_entry["serial"])
  IBPproduct_location["LOCATIONPRODUCTDEL"]=" "
  #IBPproduct_location["VRFKZ"]="X"

  IBPproduct_location.to_csv(filename+"product_location.csv",sep=";",index=False,header=True)


def write_singel_string_wares_attribute_to_csv_as_IBPproduct(wares):
  IBPmaterial = pd.DataFrame(columns=['PRDID','PRODDESC','PRODGROUP','PRODTYPE','PRODUCTDEL','PRODUCTHIERARCHY','UOMID'])

  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())

  wares_amount_data.columns = ['ware', 'amount']
  IBPmaterial["PRDID"]=wares_amount_data["ware"]
  IBPmaterial["UOMID"]="ST"
  IBPmaterial['PRODUCTDEL']=" "
  IBPmaterial["PRODDESC"]="no desc lang E"

  #IBpmaterialdes = pd.DataFrame(columns=["MANDT","LOGSYS","RESID","SPRAS","KTEXT"])
  #IBpmaterialdes["RESID"]=IBPmaterial["MATNR"]
  #IBpmaterialdes["SPRAS"]="E"
  #IBpmaterialdes["MANDT"]="001"
  #IBpmaterialdes["LOGSYS"]="WDL"
  #IBpmaterialdes.to_csv("materialsdesc.csv",sep=";",index=False,header=False)
  IBPmaterial.to_csv("PRODUCT.csv",sep=";",index=False,header=True)

def write_one_buildings_entry_to_IBPstock(building_entry,filename):
  IBPstocks = pd.DataFrame(columns=['MANDT','DELKZ','LOGSYS','MATNR','LOC_WERKS','LGORT','PLAAB','BLANR','LOC_LIFNR','LOC_KUNNR','CHARG','TST01','VRFKZ','MNG01'])
  wares=building_entry["wares"]
  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())
  wares_amount_data.columns = ['ware', 'amount']
  IBPstocks["MATNR"]=wares_amount_data["ware"]
  IBPstocks["MNG01"]=wares_amount_data["amount"]
  IBPstocks['LOC_WERKS']=str(building_entry["serial"])
  IBPstocks["VRFKZ"]="X"
  IBPstocks['MANDT']="001"
  IBPstocks['LOGSYS']="WDL"
  IBPstocks.to_csv(filename+"stocks.csv",sep=";",index=False,header=False)

def write_one_buildings_orders_to_IBPorders(orders):
  IBPorders = pd.DataFrame(columns=['MANDT','LOGSYS','DELNR','DELPS','DELET','DELKZ','MATNR','LOC_WERKS',
  'CHARG','PLAAB','PLANR','LGORT','LOC_LIFNR','LOC_KUNNR','TST01','VRFKZ','MNG01','MNG02',
  'MNG03','TST02','FIX01','LOC_WERKS_FROM','VERID','DELVR','AUFVR','POSVR',
  'INFNR','EKORG','ESOKZ','MNG04','DEPLOYMENT','MOT_ID','EINVR','DPS_TST','PP_STAGE','REL_FIXED_STATUS'])
  IBPorders['MANDT']="001"
  IBPorders['LOGSYS']="WDL"
  IBPorders['DELNR']=orders.serial.apply(str)+orders.name.apply(str)+orders.time.apply(str)+orders.requested_Ware.apply(str)+orders.amount.apply(str)
  IBPorders.to_csv("IBPorders.csv")

  
def rewrite_wares_to_one_simgle_ware(wares,ware):
  return wares[ware]

def write_one_product_pds_header(data,product):
  print(data["wares"].apply(rewrite_wares_to_one_simgle_ware,ware=product))


  


In [2]:
data = read_data_from_files(directory)


In [3]:
a=map_to_real_time(read_data_from_files(directory))


In [22]:
print(a[0])

        serial                        name                    time  \
0          961     atlanteans_headquarters 2021-01-01 00:27:01.600   
1          961     atlanteans_headquarters 2021-01-01 00:54:03.200   
2          961     atlanteans_headquarters 2021-01-01 01:07:34.000   
3          961     atlanteans_headquarters 2021-01-01 01:21:04.800   
4          961     atlanteans_headquarters 2021-01-01 01:48:06.400   
...        ...                         ...                     ...   
721479   69187        atlanteans_warehouse 2022-01-01 00:14:43.800   
721480  131182  atlanteans_foresters_house 2022-01-01 00:15:21.100   
721481  131182  atlanteans_foresters_house 2022-01-01 00:15:21.100   
721482   69187        atlanteans_warehouse 2022-01-01 00:25:45.400   
721483     961     atlanteans_headquarters 2022-01-01 00:27:01.600   

        ware_economy_serial  worker_economy_serial position  \
0                         2                      3  (18,59)   
1                         2      

In [4]:
string=a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ]["wares"][0]
write_singel_string_wares_attribute_to_csv_as_IBPproduct(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ]["wares"][0])

In [5]:
write_one_buildings_entry_to_IBPstock(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ].iloc[0],str(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ].iloc[0]["serial"]))

In [6]:
write_product_location(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ].iloc[0],str(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ].iloc[0]["serial"]))

In [7]:
string=a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ]["wares"][0]
string = re.sub(r".*wares\:", "", string)

In [8]:
Dict = json.loads(string)
fuck = pd.DataFrame.from_dict(Dict.items())
fuck.columns = ['ware', 'amount']

In [9]:
orders=read_orders_data_from_files(directory)
print((orders[0]["time"]))
orders_maped = map_to_real_time(orders)

0            8579
1            8579
2            8579
3            8579
4            8779
           ...   
12262    19447027
12263    19447027
12264    19447227
12265    19447227
12266    19448408
Name: time, Length: 12267, dtype: int64


In [10]:
#print((orders[0]["time"]))
print(orders_maped[0].columns)

Index(['serial', 'name', 'time', 'requested_Ware', 'amount'], dtype='object')


In [11]:

orders[0].rename(columns={'building_serial':'serial'}, inplace=True)
merged_data = a[0].merge(orders[0], on=["serial","time"], how='left')


In [12]:
print(merged_data[~merged_data["requested_Ware"].isnull()].columns)

Index(['serial', 'name_x', 'time', 'ware_economy_serial',
       'worker_economy_serial', 'position', 'reserved_by_worker', 'workers',
       'is_mine', 'is_port', 'needs_seafaring', 'needs_water_ways', 'passable',
       'is_production_site', 'is_warehouse', 'is_market', 'ware_priorities',
       'wares', 'input_queues', 'produced_Wares', 'workinpositions', 'name_y',
       'requested_Ware', 'amount'],
      dtype='object')


In [13]:
for i in range(len(data)):
  write_series_of_location_to_csv_as_IBPlocation(data[i][["serial","name"]].drop_duplicates(subset="serial"),str(i))




In [14]:
b=data[i][["serial","name"]].drop_duplicates(subset="serial")

In [15]:
write_one_buildings_orders_to_IBPorders(orders[0])

In [16]:
a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ,"wares"]=a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ]["wares"].apply(split_wares)

In [17]:
print(type(a[0]["wares"].iloc[0]))

<class 'dict'>


In [18]:
headquarters = a[0][a[0]['wares'].str.len()==110]
print(type(headquarters["wares"].iloc[0]))

<class 'dict'>


In [19]:
#headquarters.loc[:,"wares"]= headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware="log")

In [20]:
print(headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware="log").apply(int).apply(filter_one_or_zero))

0         1
1         1
2         1
3         1
4         1
         ..
721445    0
721471    0
721475    0
721476    0
721483    0
Name: wares, Length: 42228, dtype: int64


In [21]:
copy= pd.DataFrame(columns=["serial","name","time","wares"])
copy["wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware="log").apply(int).apply(filter_one_or_zero)
#copy.wares= copy.wares.apply(filter_one_or_zero)
copy[["serial","name","time"]] = headquarters[["serial","name","time"]]
#copy.iloc[:,"wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware=i)
sequence =copy.wares.astype(str).str.cat()
#print (sequence)
indices_object = re.finditer(pattern='01', string=sequence)
indices0_1 = [index.start() for index in indices_object]
indices_object = re.finditer(pattern='10', string=sequence)
indices1_0 = [index.start() for index in indices_object]
zeitintervalls =[]
if min(indices1_0) < min(indices0_1):
  indices = sorted(indices1_0.extend(indices0_1))
  if (len(indices)%2 != 0):
    indices.insert(0,0)




print(indices0_1,indices1_0)

TypeError: 'NoneType' object is not iterable

SyntaxError: invalid syntax (1900970059.py, line 1)

In [ ]:
data_to_be_written_to_csv = pd.DataFrame(columns=['MANDT','LOGSYS','LOCID'
,'PRDID','PRODUCTIONVERSION','ASSEMBLYSCRAP','BOMEXPLOSIONRULE'
,'OUTPUTCOEFFICIENT','PDSDEL','PDSPRIO','PMAXLOTSIZE','PMINLOTSIZE'
,'PRODUCTIONVERSIONDESCR','PROUNDING','PVALIDFR','PVALIDTO'])
for i in headquarters["wares"].iloc[0].keys():
  print(headquarters["wares"])
  copy= pd.DataFrame(columns=["serial","name","time","wares"])
  copy["wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware=i)
  copy.loc[:,"wares"]= copy["wares"].apply(filter_one_or_zero)
  copy[["serial","name","time"]] = headquarters[["serial","name","time"]]
  #copy.iloc[:,"wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware=i)
  print(copy.wares)
  sequence =copy.wares.astype(str).str.cat()
  print(sequence)
  data_to_be_written_to_csv = pd.DataFrame(columns=['MANDT','LOGSYS','LOCID'
  ,'PRDID','PRODUCTIONVERSION','ASSEMBLYSCRAP','BOMEXPLOSIONRULE'
  ,'OUTPUTCOEFFICIENT','PDSDEL','PDSPRIO','PMAXLOTSIZE','PMINLOTSIZE'
  ,'PRODUCTIONVERSIONDESCR','PROUNDING','PVALIDFR','PVALIDTO'])
  data_to_be_written_to_csv["PRDID"]=i


0         {'balsa': '0', 'log': '20', 'rope': '0', 'rubb...
1         {'balsa': '0', 'log': '20', 'rope': '0', 'rubb...
2         {'balsa': '0', 'log': '20', 'rope': '0', 'rubb...
3         {'balsa': '0', 'log': '20', 'rope': '0', 'rubb...
4         {'balsa': '0', 'log': '20', 'rope': '0', 'rubb...
                                ...                        
385356    {'balsa': '0', 'log': '0', 'rope': '0', 'rubbe...
385406    {'balsa': '0', 'log': '0', 'rope': '0', 'rubbe...
385435    {'balsa': '0', 'log': '0', 'rope': '0', 'rubbe...
385473    {'balsa': '0', 'log': '0', 'rope': '0', 'rubbe...
385530    {'balsa': '0', 'log': '0', 'rope': '0', 'rubbe...
Name: wares, Length: 31285, dtype: object
0         1
1         1
2         1
3         1
4         1
         ..
385356    1
385406    1
385435    1
385473    1
385530    1
Name: wares, Length: 31285, dtype: int64
...
0         {'balsa': '0', 'log': '20', 'rope': '0', 'rubb...
1         {'balsa': '0', 'log': '20', 'rope': '0', 'rubb...
2

KeyboardInterrupt: 

In [ ]:
copy= pd.DataFrame(columns=["serial","name","time","wares"])
copy["wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware="log")
copy[["serial","name","time"]] = headquarters[["serial","name","time"]]
#copy.iloc[:,"wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware=i)
sequence =copy.wares.astype(str).str.cat()
indices_object = re.finditer(pattern='01', string=sequence)
indices = [index.start() for index in indices_object]

print(indices)
#print(sequence)
#print(sequence.find('6'))

[73, 12892, 13051, 13068, 13121, 13180, 13560, 13562, 13564, 13566, 13568, 13570, 13572, 13574, 13576, 13578, 13580, 13582, 13584, 13586, 13588, 13590, 13592, 13594, 13596, 13598, 13600, 13602, 13604, 13643, 13645, 13647, 13649, 13651, 13653, 13655, 13657, 13659, 13661, 13663, 13665, 13667, 13669, 13671, 13673, 13675, 13677, 13687, 13689, 13691, 13693, 13695, 13697, 13699, 13701, 13703, 13705, 13707, 13709, 13711, 13713, 13715, 13717, 13719, 13721, 13723, 13725, 13727, 13729, 13731, 13733, 13735, 13737, 13886, 13888, 13890, 13892, 13894, 13896, 13898, 13900, 13902, 13904, 13906, 13908, 13910, 13912, 13914, 14008, 14010, 14012, 15182, 15240, 16093, 16188, 16216, 16288, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 15232, 15234, 15236, 15238, 651, 13679, 13851]


In [ ]:
print(headquarters.loc[73,"wares"]["log"])

18


In [ ]:
filter_one_or_zero(188)

1